In [20]:
import pandas as pd
import numpy as np
import os
import re

## Load, clean, and format CGM Data:

Function to load and clean CGM data for one patient:

In [21]:
def clean_and_load_dexcom(pathname):
    """
    Load and clean a Dexcom CSV file.
    
    Filters for EGV (Estimated Glucose Values),
    renames columns to 'Timestamp' and 'Value',
    and adds a 'Source' column labeled 'Dexcom'.
    
    Parameters:
        pathname (str): Path to the Dexcom CSV file.
        
    Returns:
        pd.DataFrame: Cleaned DataFrame with 'Timestamp', 'Value', and 'Source'.
    """
    df = pd.read_csv(pathname)

    # Keep only relevant columns
    df = df[['Timestamp (YYYY-MM-DDThh:mm:ss)', 'Glucose Value (mg/dL)', 'Event Type']]

    # Filter to only EGV (Estimated Glucose Values)
    df = df[df['Event Type'] == 'EGV']

    # Drop the Event Type column
    df = df.drop('Event Type', axis=1)

    # Rename columns
    df = df.rename(columns={
        'Timestamp (YYYY-MM-DDThh:mm:ss)': 'timestamp',
        'Glucose Value (mg/dL)': 'value'
    })


    return df


Iterate over each patient, load their CGM data with clean_and_load_dexcom(), add patient_id column, and concat into one master dataframe:

In [22]:
def load_all_dexcom(folder_path):
    """
    Load and combine all Dexcom CSV files (Dexcom_001.csv through Dexcom_016.csv)
    from a folder into a single cleaned DataFrame.

    Adds a 'patient_id' column to each row.

    Parameters:
        folder_path (str): Path to the folder containing the Dexcom CSV files.

    Returns:
        pd.DataFrame: Combined DataFrame with 'Timestamp', 'Value', 'Source', and 'patient_id'.
    """
    all_dfs = []

    for i in range(1, 17):
        filename = f"Dexcom_{i:03d}.csv"
        filepath = os.path.join(folder_path, filename)
        
        df = clean_and_load_dexcom(filepath)
        df['patient_id'] = i
        all_dfs.append(df)

    combined_df = pd.concat(all_dfs, ignore_index=True)
    return combined_df


In [23]:
dexcom_data = load_all_dexcom('data/dexcom')

In [25]:
dexcom_data

,timestamp,value,patient_id
0,2020-02-13 17:23:32,61.0,1
1,2020-02-13 17:28:32,59.0,1
2,2020-02-13 17:33:32,58.0,1
3,2020-02-13 17:38:32,59.0,1
4,2020-02-13 17:43:31,63.0,1
...,...,...,...
36893,2020-07-24 09:58:05,108.0,16
36894,2020-07-24 10:03:05,108.0,16
36895,2020-07-24 10:08:05,106.0,16
36896,2020-07-24 10:13:05,102.0,16


## Load, clean, and format Food Log data:

Function to load and clean food log data for one patient. Retain sugar, time, and searched food columns. 

Uses regex to find values in 'time' column that aren't formatted as 'hh:mm:ss', and replaces them with the most recent valid time.

The searched food column is used over the logged food column because it provides a more accurate description of the item the patient ate. If the value in the searched food column is null, default to logged food column. This could be good if we want to analyze individual meals. On the other hand, maybe we want to aggregate based on popular foods in the 'logged_foods' column instead to get a greater idea of which meals cause which certain changes in glucose levels. Some values in logged foods seem to be relatively general with names like 'standard breakfast', which might support aggregation. But after looking further, it seems that there is still a lot of variability within logged foods, so this might not be achievable. 

In [16]:
def clean_and_load_food_log(pathname):
    """
    Load and clean a food log CSV file.

    Fixes malformed time strings by using the previous valid value,
    parses mixed-format dates and times robustly,
    and returns a cleaned DataFrame with sugar values and food descriptions.

    Parameters:
        pathname (str): Path to the food log CSV file.

    Returns:
        pd.DataFrame: Cleaned DataFrame with 'Timestamp', 'Value', and 'Food'.
    """
    df = pd.read_csv(pathname)

    # Extract date and time columns
    date_col = df.iloc[:, 0].astype(str)
    time_col = df.iloc[:, 1].astype(str)

    # Regex pattern for valid HH:MM:SS
    time_pattern = re.compile(r"^\d{2}:\d{2}:\d{2}$")

    # Fix malformed times
    fixed_times = []
    prev_time = "00:00:00"
    for t in time_col:
        if time_pattern.match(t):
            prev_time = t
        fixed_times.append(prev_time)

    # Parse combined date and fixed time
    timestamp_series = pd.to_datetime(date_col + ' ' + fixed_times, errors='coerce')

    # Drop rows with unparseable timestamps
    df = df[timestamp_series.notna()].copy()
    df['timestamp'] = timestamp_series[timestamp_series.notna()]

    # Extract sugar value (index 10) and food description (index 7)
    df['value'] = pd.to_numeric(df.iloc[:, 10], errors='coerce')
    df['food'] = df.iloc[:, 7].astype(str)

    return df[['timestamp', 'value', 'food']]


Iterate over each patient, load their food log data with clean_and_load_food_log(), add patient_id column, and concat into one master dataframe:

In [17]:
def load_all_food_logs(folder_path):
    """
    Load and combine all Food_Log_0xx.csv files from a folder into one cleaned DataFrame.
    
    Adds a 'patient_id' column to each row.

    Parameters:
        folder_path (str): Path to the folder containing the food log CSV files.

    Returns:
        pd.DataFrame: Combined DataFrame with 'Timestamp', 'Value', and 'patient_id'.
    """
    all_dfs = []

    for i in range(1, 17):
        filename = f"Food_Log_{i:03d}.csv"
        filepath = os.path.join(folder_path, filename)

        df = clean_and_load_food_log(filepath)
        df['patient_id'] = i
        all_dfs.append(df)

    combined_df = pd.concat(all_dfs, ignore_index=True)
    return combined_df



In [18]:
food_log_data = load_all_food_logs('data/food_logs')

In [19]:
food_log_data

,timestamp,value,food,patient_id
0,2020-02-13 18:00:00,1.7,Strawberry Smoothie,1
1,2020-02-13 20:30:00,0.0,chicken leg,1
2,2020-02-13 20:30:00,1.2,Asparagus,1
3,2020-02-14 07:10:00,NaN,(Natrel) Lactose Free 2% Partly Skimmed Milk,1
4,2020-02-14 07:10:00,NaN,"(Kellogg's) Frosted Flakes, Cereal",1
...,...,...,...,...
1416,2020-02-26 00:00:00,0.0,nan,16
1417,2020-02-27 00:00:00,1.0,nan,16
1418,2020-02-27 00:00:00,3.9,nan,16
1419,2020-02-27 00:00:00,11.0,nan,16
